In [1]:
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# Load the OpenAI API key from an environment variable
openai_api_key = os.environ["OPENAI_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv('LANGCHAIN_API_KEY')

# Initialize the ChatOpenAI model with the configured parameters
llm = ChatOpenAI(
    openai_api_key=openai_api_key,
    model_name="gpt-3.5-turbo",
    temperature=0.6,
    top_p=0.8,
    presence_penalty=1.5,  
    frequency_penalty=1.2,
)

d:\ProjectsAI\flashcard-generator\env\lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! top_p is not default parameter.
                top_p was transferred to model_kwargs.
                Please confirm that top_p is what you intended.
  warnings.warn(
d:\ProjectsAI\flashcard-generator\env\lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! presence_penalty is not default parameter.
                presence_penalty was transferred to model_kwargs.
                Please confirm that presence_penalty is what you intended.
  warnings.warn(
d:\ProjectsAI\flashcard-generator\env\lib\site-packages\langchain_core\utils\utils.py:159: UserWarning: WARNING! frequency_penalty is not default parameter.
                frequency_penalty was transferred to model_kwargs.
                Please confirm that frequency_penalty is what you intended.
  warnings.warn(


In [3]:
# Loading PDF files
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter

In [4]:
#Extract title and content from pdf files
def extract_PDF_data(pdf_path):
    base_name=os.path.basename(pdf_path)
    pdf_title=os.path.splitext(base_name)[0]
    pdf_title = pdf_title.replace(' ', '_').replace(':', '').replace('/', '_')
    pdf_reader = PdfReader(pdf_path)
    return pdf_reader, pdf_title

In [5]:
# File path
pdf_path=r'D:\ProjectsAI\flashcard-generator\data\curso-274850-aula-03-prof-herbert-almeida-f315-simplificado.pdf'
pdf_reader, pdf_title=extract_PDF_data(pdf_path)

In [6]:
# Content handle
raw_text = ''
# Start from page 4 (index 3) instead of the first page
for i, page in enumerate(pdf_reader.pages):
    if i < 3:  # Skip the first three pages
        continue
    if i >= 99:  # Stop importing pages after the 72nd page
        break
    
    content = page.extract_text()
    if content:
        if "RESUMO ESTRATÉGICO" in content or "QUESTÕES PARA FIXAÇÃO" in content:
            break  # Stop importing pages when this text is found
        raw_text += content


In [7]:
print(raw_text)

Os contratos  são acordos de vontades, com objetivo determinado, no qual as partes ajustam as respectivas 
obrigações. Por exemplo: você e um pintor (partes) podem combinar a realização da pintura da sua casa 
(objetivo), ajustando que você pagará o valor X e ele realizará o serviço de pintura (obrigações).  
A administração pública também pode figurar como parte nas relações contratuais, buscando atender às 
necessidades da própria administração ou ainda atender às necessidades da população. Portanto, existem 
diversos contratos em que a administração pública fi gurará como parte da relação. Genericamente falando, 
todo contrato em que a administração seja parte será denominado contrato da administração . 
Perceba, portanto, que até agora não mencionamos características especiais para estes contratos, exceto 
o fato de a administração constar como parte. Assim, entenda que “contrato da administração” é o gênero , 
que comporta todos os contratos em que a administração figura como part

In [23]:
#Production text chunks
text_splitter=CharacterTextSplitter(
    separator='\n',
    chunk_size=7000,
    length_function=len,
)
text_chunks=text_splitter.split_text(raw_text)

In [24]:
len(text_chunks)

32

In [18]:
RESPONSE_JSON_STRUCTURE = {
  "1": {
    "pergunta": "Sua pergunta sobre o conceito do texto",
    "resposta": "A resposta para a sua pergunta"
  },
  "2": {
    "pergunta": "Sua pergunta sobre outro conceito",
    "resposta": "A resposta para a sua segunda pergunta"
  },
  # // ... e assim por diante para cada conceito identificado
}

In [19]:
prompt_template_flashcards = """
Texto Fornecido: {text}
Como especialista na criação de flashcards educacionais, sua tarefa é extrair os principais conceitos do texto fornecido e formular perguntas e respostas concisas.
Passos:
1. Identifique os conceitos principais ou temas do texto fornecido.
2. Formule uma pergunta clara e concisa para cada conceito principal ou tema, focando em facilitar a compreensão do texto.
3. Forneça uma resposta direta e sucinta para cada pergunta.
4. Retorne os flashcards gerados no formato JSON especificado:
{response_json}
Observação: A estrutura de resposta deve conter apenas as perguntas e respostas essenciais que refletem os conceitos principais.
"""

In [20]:
card_generation_prompt=PromptTemplate(
    input_variables=['text','response_json'],
    template=prompt_template_flashcards
)

card_chain=LLMChain(llm=llm,prompt=card_generation_prompt, output_key='cards',verbose=True)

In [21]:
#Get text chunks, file name and convert into flashcards
def card_generator(text_chunks,filename):
    card_table_data = []
    response_json = json.dumps(RESPONSE_JSON_STRUCTURE)
    
    for chunk in text_chunks:
        response = card_chain({'text': chunk, 'response_json': response_json})
        card_str = response.get('cards')
        
    #for chunk in range(0, 5):
        #response = card_chain({'text': text_chunks[chunk], 'response_json': response_json})
        #card_str = response.get('cards')

        # Try-except block to catch JSON parsing errors
        try:
            card_dict = json.loads(card_str)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON from chunk: {e}")
            continue  # Skip this chunk and proceed with the next one

        for key, value in card_dict.items():
            pergunta = value.get('pergunta')
            resposta = value.get('resposta')
            if pergunta and resposta:  # Ensure both pergunta and resposta are not None
                card_table_data.append({'pergunta': pergunta, 'resposta': resposta})
            else:
                print(f"Missing 'pergunta' or 'resposta' in chunk: {chunk}")

    df_cards = pd.DataFrame(card_table_data)
    df_cards.to_csv(f'{filename}.csv', index=False)

In [22]:
card_generator(text_chunks,pdf_title)



> Entering new LLMChain chain...
Prompt after formatting:

Texto Fornecido: Os contratos  são acordos de vontades, com objetivo determinado, no qual as partes ajustam as respectivas 
obrigações. Por exemplo: você e um pintor (partes) podem combinar a realização da pintura da sua casa 
(objetivo), ajustando que você pagará o valor X e ele realizará o serviço de pintura (obrigações).  
A administração pública também pode figurar como parte nas relações contratuais, buscando atender às 
necessidades da própria administração ou ainda atender às necessidades da população. Portanto, existem 
diversos contratos em que a administração pública fi gurará como parte da relação. Genericamente falando, 
todo contrato em que a administração seja parte será denominado contrato da administração . 
Perceba, portanto, que até agora não mencionamos características especiais para estes contratos, exceto 
o fato de a administração constar como parte. Assim, entenda que “contrato da administração” é o gên